# Churn Model Scoring (Batch Prediction)

This notebook scores all users using the trained churn model and stores predictions in CHURN_PREDICTIONS table.

## Steps:
1. Load trained model from OML Datastore
2. Load user features from CHURN_USER_FEATURES view
3. Score all users (batch prediction)
4. Store predictions in CHURN_PREDICTIONS table

In [ ]:
%python

# Cell 1: Import and Setup
import oml
import pandas as pd
import numpy as np
from datetime import datetime
import oracledb
import os

print("=" * 60)
print("Churn Model Scoring (Batch Prediction)")
print("=" * 60)
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Check OML connection
if oml.isconnected():
    print("✓ OML connected")
else:
    print("⚠️  OML not connected")

In [ ]:
%python

# Cell 2: Load Model from OML Datastore
model_name = 'churn_xgboost_v1'

print(f"Loading model '{model_name}' from OML datastore...")
try:
    model_dict = oml.ds.load(model_name)
    xgb_model = model_dict['model']
    print(f"✓ Model loaded successfully")
    print(f"  Model type: {type(xgb_model)}")
except Exception as e:
    print(f"❌ ERROR: Failed to load model: {e}")
    print("   Make sure you've trained and saved the model first!")
    raise

In [ ]:
%python

# Cell 3: Load User Features
print("Loading user features from CHURN_USER_FEATURES view...")
user_features_oml = oml.sync(view='CHURN_USER_FEATURES')
user_features_pd = user_features_oml.pull()

print(f"✓ Loaded {len(user_features_pd):,} user profiles")

# Get USER_ID and features
user_ids = user_features_pd['USER_ID'].copy()
feature_cols = [col for col in user_features_pd.columns if col != 'USER_ID']
X_users = user_features_pd[feature_cols].copy()

print(f"✓ Features: {len(feature_cols)}")

# Clean data
print("\nCleaning data...")
for col in feature_cols:
    if pd.api.types.is_numeric_dtype(X_users[col]):
        X_users[col] = X_users[col].replace([np.inf, -np.inf], np.nan)
        X_users[col] = X_users[col].fillna(0)

print("✓ Data cleaned")

In [ ]:
%python

# Cell 4: Score Users
# Push to OML
print("Pushing user features to OML...")
X_users_oml = oml.push(X_users)
X_users_oml_features = X_users_oml[feature_cols]

print(f"✓ Features pushed: {X_users_oml_features.shape}")

# Get predictions
print("Generating predictions...")
y_pred_proba_oml = xgb_model.predict_proba(X_users_oml_features)

# Convert to pandas
y_pred_proba_pd = y_pred_proba_oml.pull()
if isinstance(y_pred_proba_pd, pd.DataFrame):
    if 1 in y_pred_proba_pd.columns:
        churn_probabilities = y_pred_proba_pd[1].values
    elif len(y_pred_proba_pd.columns) == 2:
        churn_probabilities = y_pred_proba_pd.iloc[:, 1].values
    else:
        churn_probabilities = y_pred_proba_pd.values.flatten()
else:
    churn_probabilities = np.array(y_pred_proba_pd)

print(f"✓ Generated {len(churn_probabilities):,} predictions")
print(f"  Average churn probability: {churn_probabilities.mean():.4f}")
print(f"  Max churn probability: {churn_probabilities.max():.4f}")
print(f"  Min churn probability: {churn_probabilities.min():.4f}")

In [ ]:
%python

# Cell 5: Store Predictions in Database
# Note: This cell requires database connection
# You may need to configure connection parameters

# Calculate predictions
threshold = 0.5
predicted_labels = (churn_probabilities >= threshold).astype(int)
risk_scores = (churn_probabilities * 100).astype(int).clip(0, 100)
model_version = 'churn_xgboost_v1'
prediction_date = datetime.now()

print("=" * 60)
print("Storing Predictions")
print("=" * 60)

# Database connection (adjust as needed)
# For OML Notebooks, you may need to use oml.connect() or configure connection
# This example assumes you have connection configured

try:
    # Try to get connection from environment or OML
    # Adjust this based on your OML Notebooks setup
    connection = None  # You'll need to configure this
    
    if connection is None:
        print("⚠️  Database connection not configured")
        print("   Predictions calculated but not stored")
        print("   Configure connection in this cell to store predictions")
    else:
        cursor = connection.cursor()
        
        # Clear existing predictions
        try:
            cursor.execute("TRUNCATE TABLE OML.CHURN_PREDICTIONS")
            print("✓ Cleared existing predictions")
        except Exception as e:
            print(f"⚠️  WARNING: Could not truncate table: {e}")
        
        # Insert predictions
        print(f"\nInserting {len(user_ids):,} predictions...")
        
        insert_sql = """
            INSERT INTO OML.CHURN_PREDICTIONS (
                USER_ID,
                PREDICTED_CHURN_PROBABILITY,
                PREDICTED_CHURN_LABEL,
                RISK_SCORE,
                MODEL_VERSION,
                PREDICTION_DATE
            ) VALUES (:1, :2, :3, :4, :5, :6)
        """
        
        data_tuples = []
        for i in range(len(user_ids)):
            data_tuples.append((
                str(user_ids.iloc[i]),
                float(churn_probabilities[i]),
                int(predicted_labels[i]),
                int(risk_scores[i]),
                model_version,
                prediction_date
            ))
        
        cursor.executemany(insert_sql, data_tuples)
        connection.commit()
        print(f"✓ Successfully inserted {len(data_tuples):,} predictions")
        
        # Verify
        cursor.execute("SELECT COUNT(*) FROM OML.CHURN_PREDICTIONS")
        count = cursor.fetchone()[0]
        print(f"✓ Verified: {count:,} rows in CHURN_PREDICTIONS table")
        
        # Summary statistics
        cursor.execute("""
            SELECT 
                COUNT(*) AS TOTAL,
                SUM(CASE WHEN PREDICTED_CHURN_LABEL = 1 THEN 1 ELSE 0 END) AS AT_RISK,
                AVG(PREDICTED_CHURN_PROBABILITY) * 100 AS AVG_RISK_SCORE,
                AVG(RISK_SCORE) AS AVG_RISK_SCORE_INT
            FROM OML.CHURN_PREDICTIONS
        """)
        stats = cursor.fetchone()
        total, at_risk, avg_prob, avg_risk = stats
        
        print(f"\nPrediction Summary:")
        print(f"  Total users: {total:,}")
        print(f"  At-risk users: {at_risk:,} ({at_risk/total*100:.2f}%)")
        print(f"  Average risk score: {avg_risk:.1f}%")
        
        cursor.close()
        
except Exception as e:
    print(f"❌ ERROR: Failed to store predictions: {e}")
    import traceback
    traceback.print_exc()